In [1]:
import numpy as np
import pandas as pd
np.random.seed(42)

# CSV-Datei: fehlende Werte, NAN
Es kommt häufig vor, dass in Datensätzen Werte fehlen. Zum Beispiel fehlende Sensorwerte oder nicht als Zahl intertpretierbare Zahlen. 

## Was sind NAN-Werte?
NaN-Werte (kurz für Not a Number) sind spezielle Werte in Daten, die verwendet werden, um fehlende oder undefinierte Daten in einem Datensatz darzustellen. Sie sind ein zentraler Bestandteil von Datenanalyse- und Datenverarbeitungsbibliotheken wie Pandas. 

Numpy bietet einen numerischen Datentyp `nan` an, den wir nutzen können. Im folgenden Beispiel erstellen wir einen DataFrame aus einem Dictionary mit einem NaN-Wert. Uns fällt auf, dass der Wert NaN ein Float-Wert ist (nach ISO 754).

In [2]:
type(np.nan)

float

In [3]:
# erstelle ein Dict mit NAN-Werten
d = {
    "a": [1, 2, 3, 4],
    "b": [4, 5, np.nan, 6],
}
features = pd.DataFrame(d)
features, features.b

(   a    b
 0  1  4.0
 1  2  5.0
 2  3  NaN
 3  4  6.0,
 0    4.0
 1    5.0
 2    NaN
 3    6.0
 Name: b, dtype: float64)

### Probleme beim arithmetischen Operationen
Bei vielen Daten würde es u.U. gar nicht auffallen, dass sich NaN-Werte eingeschlichen hätten. Würden wir zum Beispiel die Summe aller Werte einer Spalte berechnen, wäre das Ergebnis allerdings kein NaN-Wert, sondern der NaN-Wert wird einfach ignoriert! 

Generell gilt: bei arithmetischen Operationen wie `sum()` werden NaN-Werte als 0 aufgefasst.

In [6]:
# wir können zwar damit rechnen, Nan-Werte werden als 0 aufgefasst
features.b.sum(), features.b.cumsum(skipna=False)

(np.float64(15.0),
 0    4.0
 1    9.0
 2    NaN
 3    NaN
 Name: b, dtype: float64)

#### Cumsum
in der kummulierten Summe ergibt sich ein falscher Wert, da NaN-Werte ignoriert werden. 

Mit `skipna` lässt sich dieses Verhalten ändern. Dann findet tatsächlich eine Addition statt, aber es gilt: 2 + Nan = NaN.

In [3]:
# cumsum von Spalte b

## Beispiel-Datei Sensordaten
Wir wollen Sensordaten einer sensordata.csv importieren und prüfen, wie wir mit fehlenden Werten umgehen können.

### Datei einlesen
Probleme: Nicht vorhande Werte werden als NaN (Not a number) abgebildet und die Werte der ersten Zeile der kopflosen CSV-Datei werden als Spaltennamen verwendet. Wenn wir beim Einlesen `header=None` setzen, werden numerische Spaltennamen vergeben (Int64Index).

In [7]:
file_name = "../data/sensordata.csv"
df = pd.read_csv(file_name, header=None)
df.head(3)

,0,1,2
0,NaN,1218.00,1210.18
1,NaN,NaN,868.44
2,1117.92,636.51,NaN


### 1. Spaltennamen anpassen
Die 3 Spalten bezeichnen die 3 Sensoren A, B, C. Wir wollen die Spaltennamen jetzt umbenennen.

In [8]:
# Spaltennamen angeben
df.columns = ["A", "B", "C"]
df

,A,B,C
0,NaN,1218.00,1210.18
1,NaN,NaN,868.44
2,1117.92,636.51,NaN
3,922.26,1400.81,NaN
4,2043.03,1103.48,1143.08
...,...,...,...
995,637.47,806.51,727.65
996,951.60,1535.97,147.27
997,1289.55,914.46,929.24
998,1717.64,397.15,735.26


## Fehlende Werte anzeigen lassen
Wir können uns eine boolsche Matrix mit den fehlenden Werten anzeigen lassen. Mit solchen boolschen Serien und Matritzen lässt sich später auch filtern.

In [9]:
# isna nutzen, umd eine boolsche Matrix zu erstellen
missing = df.isna()
missing

,A,B,C
0,True,False,False
1,True,True,False
2,False,False,True
3,False,False,True
4,False,False,False
...,...,...,...
995,False,False,False
996,False,False,False
997,False,False,False
998,False,False,False


### Fehlende Werte zählen
- isna(): Not Available
- isnull(): ältere Methode, die genau gleich isna() ist (deprecated)

In [11]:
# Spaltenweise NAN-Werte zählen
df.isna().sum()

A    20
B    33
C    39
dtype: int64

### Prüfen, ob in Spalte A ein wahrer Wert vorkommt (Series Methoden)

Die Methoden any, all und isnull() können ausgefüht werden, zu prüfen, ob Werte falsy oder NaN sind
- isnull: eine boolsche Serie, die wahr zeigt, wenn NaN
- any: eine boolsche Funktion, die wahr ist, wenn ein Wert der Serie wahr ist
- all: eine boolsche Funktion, die wahr ist, wenn ein Wert der Serie wahr ist

In [8]:
# any nutzen, um zu prüfen, ob in Spalte a zumindest ein wahrer Wert vorkommt (vlg. Python any)
d = {
    "a": [0, 0, 0, np.nan],
    "b": [4, 5, 1, 6],
    "c": [0, 0, 0, 0]
}

### Prüfen, ob in Spalte A alle Werte truthy sind (all)

In [9]:
# all nutzen

### 2. Zeilen mit Nan-Werten aus dem DataFrame löschen
eine mögliche Strategie, mit NaN-Werten umzugehen, ist es, Zeilen, die mindestens einen NaN-Wert enthalten, komplett zu löschen. Das ist nicht immer möglich aber bei Sensordaten, die im Millisekundentakt gemessen werden, ist das eine angemessene Strategie.

In [14]:
# DataFrame anhand der Rows-Achse löschen, wenn ein NAN-Wert in der Zeile vorkommt
df_cleaned = df.dropna(axis="rows", thresh=2)
df_cleaned.head(3)

,A,B,C
0,NaN,1218.00,1210.18
2,1117.92,636.51,NaN
3,922.26,1400.81,NaN


### 3. Mindest Anzahl an NaN-Werten pro Zeile
wir können für die Methode `dropna()` auch einen threshhold angeben, ab wievielen Nan-Werten pro Zeile die Zeile gelöscht werden soll. Im folgenden Beispiel löschen wir nur Zeilen aus dem Dataframe, die mindestens zwei NaN-Werte enthalten.

In [11]:
# threshold angeben, wieviele NaN-Werte in der Zeile vorkommen müssen


### Spalten löschen
bisher haben wir immer ganze Spalten gelöscht, wenn ein NaN-Wert in der Zeile vorkam. Wir können natürlich auch Spalten löschen. Im Beispiel bleibt nur der Index übrig, weil jede Spalte mindestens einen Nan-Wert besitzt. Grundsätzlich ist das Löschen ganzer Spalten wohl eher die Ausnahme.

In [13]:
# Zeilen löschen

### Inplace Löschen
wir müssen den Dataframe nicht zwingend umkopieren, es ist auch ein inplace-Löschen der Zeilen möglich

In [56]:
# Zeilen inplace llöschen

,0,1,2
4,2043.03,1103.48,1143.08
5,2081.55,662.27,1799.80
6,276.57,1324.42,1063.86
9,1266.55,1410.51,1803.36
10,1582.38,1281.72,1727.09
...,...,...,...
995,637.47,806.51,727.65
996,951.60,1535.97,147.27
997,1289.55,914.46,929.24
998,1717.64,397.15,735.26


## Fehlende Werte auffüllen
Wir können fehlende Werte auch mit Platzhalterwerten auffüllen. Im folgenden Beispiel füllen wir alle fehlenden Werte mit 0 auf. Die Operation lässt sich nütürlich ebenso inplace durchführen.

In [15]:
df_filled = df.fillna(value=42)
df_filled

,A,B,C
0,42.00,1218.00,1210.18
1,42.00,42.00,868.44
2,1117.92,636.51,42.00
3,922.26,1400.81,42.00
4,2043.03,1103.48,1143.08
...,...,...,...
995,637.47,806.51,727.65
996,951.60,1535.97,147.27
997,1289.55,914.46,929.24
998,1717.64,397.15,735.26


## Fehlende Werte interpolieren
Wir können fehlende Werte natürlich auch durch andere, selbst errechnete Werte ersetzen, dazu später mehr. Eine einfache Methode, fehlende Werte mit Mittelwerten vorhergehender und nachfolgender Werte zu ersetzen, ist die Methode `interpolate`. Mit dem Parameter `limit_direction` lässt sich bestimmen, ob die Interpolation von "vorne" oder hinten durchgeführt wird. Der default-Wert ist hier `forward`.

Zusätzlich kann auch noch mit `method` die Interpolations-Methode gewählt werden. Neben `linear` (default-Wert) gibt es noch `nearest`, `values`, `quadratic` und einige andere mehr. Hier sei wieder auf die Doku verwiesen:

https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/api/pandas.DataFrame.interpolate.html?highlight=interpolate#pandas.DataFrame.interpolate

In [24]:
# Fehlende Werte in  Spalten interpolieren
df["B"], df["B"].interpolate()

(0      1218.00
 1          NaN
 2       636.51
 3      1400.81
 4      1103.48
         ...   
 995     806.51
 996    1535.97
 997     914.46
 998     397.15
 999    1344.66
 Name: B, Length: 1000, dtype: float64,
 0      1218.000
 1       927.255
 2       636.510
 3      1400.810
 4      1103.480
          ...   
 995     806.510
 996    1535.970
 997     914.460
 998     397.150
 999    1344.660
 Name: B, Length: 1000, dtype: float64)

In [23]:
# Da hier die ersten beiden Werte der Spalte NaN sind, lässt sich mit forward nicht interpolieren.
# dann kann backward genutzt werden, um von hinten zu beginnen
df["A"].interpolate(limit_direction="backward")

0      1117.92
1      1117.92
2      1117.92
3       922.26
4      2043.03
        ...   
995     637.47
996     951.60
997    1289.55
998    1717.64
999    1302.95
Name: A, Length: 1000, dtype: float64